In [1]:
import pandas as pd
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, ClassificationPreset
from evidently.ui.workspace import Workspace  # Workspace used here
import pickle
from sklearn.metrics import f1_score, balanced_accuracy_score, recall_score, precision_score



# Define workspace path (local directory)
WORKSPACE_PATH = "./reporting/evidently_ui_workspace"

# Create a local workspace if it doesn't exist
workspace = Workspace.create(WORKSPACE_PATH)

# Define the paths for your reference and production data
REF_DATA_PATH = "../data/ref_data.csv"
PROD_DATA_PATH = "../data/prod_data.csv"

# Define the model path
MODEL_PATH = "../artifacts/model_xgb.pkl"

In [4]:
def preprocess_data():
    """Load and preprocess reference and production datasets."""
    ref_data = pd.read_csv(REF_DATA_PATH)
    prod_data = pd.read_csv(PROD_DATA_PATH)

    # Drop unnecessary columns
    ref_data = ref_data.loc[:, ~ref_data.columns.str.contains("^Unnamed")]
    prod_data = prod_data.loc[:, ~prod_data.columns.str.contains("^Unnamed")]

    # Rename columns
    ref_columns = [f"PCA_{i+1}" for i in range(100)] + ["target"]
    ref_data.columns = ref_columns

    # Convert PCA columns to numeric
    pca_cols = [f"PCA_{i+1}" for i in range(100)]
    for col in pca_cols:
        ref_data[col] = pd.to_numeric(ref_data[col], errors="coerce")
        prod_data[col] = pd.to_numeric(prod_data[col], errors="coerce")

    ref_data.dropna(subset=pca_cols, inplace=True)
    prod_data.dropna(subset=pca_cols, inplace=True)

    # Convert target to string
    ref_data["target"] = ref_data["target"].apply(str)
    prod_data["target"] = prod_data["target"].apply(str)

    return ref_data, prod_data, pca_cols

def load_model():
    """Load the trained model from the specified path."""
    with open(MODEL_PATH, "rb") as file:
        model = pickle.load(file)
    return model

def evaluate_model(ref_data, prod_data, pca_cols, model):
    """Generate predictions and evaluate model performance."""
    ref_data["prediction"] = model.predict(ref_data[pca_cols])
    prod_data["prediction"] = model.predict(prod_data[pca_cols])

    # Compute evaluation metrics
    metrics = {
        "F1 Score": f1_score(ref_data["target"], ref_data["prediction"], average="weighted"),
        "Balanced Accuracy": balanced_accuracy_score(ref_data["target"], ref_data["prediction"]),
        "Recall (Rappel)": recall_score(ref_data["target"], ref_data["prediction"], average="weighted"),
        "Precision": precision_score(ref_data["target"], ref_data["prediction"], average="weighted"),
    }

    print("\n[INFO] Model Performance on Reference Data:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    return ref_data, prod_data

def build_performance_dashboard(ref_data, prod_data, pca_cols):
    """Create and save the Evidently performance dashboard."""
    column_mapping = ColumnMapping(
        target="target",
        prediction="prediction",
        numerical_features=pca_cols,
    )

    # Create Evidently report
    report = Report(metrics=[ClassificationPreset()])
    report.run(reference_data=ref_data, current_data=prod_data, column_mapping=column_mapping)

    # Add to workspace
    project = workspace.create_project("Model Performance Project")
    workspace.add_report(project.id, report)

    # Save as HTML
    report.save_html("./model_performance_dashboard.html")
    print("[INFO] Model performance dashboard saved: model_performance_dashboard.html")



In [5]:
# Call the function to generate the report
    # Load and preprocess data
ref_data, prod_data, pca_cols = preprocess_data()

# Load the model
model = load_model()

# Evaluate model performance
ref_data, prod_data = evaluate_model(ref_data, prod_data, pca_cols, model)

# Create performance dashboard
build_performance_dashboard(ref_data, prod_data, pca_cols)

c:\Users\abdennour\.conda\envs\sign-language-mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:00:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\abdennour\.conda\envs\sign-language-mlops\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:00:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device or

ValueError: Mix of label input types (string and number)